In [1]:
import dotenv

from modifier import Modifier, default_faker
from sql import load_table, dataframe_to_pg_bulk_update

In [2]:
dotenv.load_dotenv()

True

# Load data

In [3]:
user = load_table("user")
user.describe()

,age
count,2.000000
mean,32.500000
std,28.991378
min,12.000000
25%,22.250000
50%,32.500000
75%,42.750000
max,53.000000


# Process data

## how to use ?
In short
- just chain all the operations
- get the result at the end

In [4]:
def random_std() -> str:
    random_number = default_faker.random_number(5, True)
    return f"STD{random_number}"

In [5]:
Modifier(user).randomize_first_name("first_name").randomize_last_name("last_name").apply_function("ref", random_std).result()

,id,ref,first_name,last_name,age
0,123,STD20240,Glenn,Middleton,12
1,321,STD18909,Lisa,Walker,53


# Create sql

In [6]:
sql = dataframe_to_pg_bulk_update(user, "user", "id")

In [7]:
print(sql)

UPDATE user
SET
    ref = data.ref,
    first_name = data.first_name,
    last_name = data.last_name,
    age = data.age
FROM (
    VALUES 
    ('123', 'STD20240', 'Glenn', 'Middleton', 12),
    ('321', 'STD18909', 'Lisa', 'Walker', 53)
) AS data(id, ref, first_name, last_name, age)
WHERE user.id = data.id;
